In [1]:
import numpy as np
from sklearn.metrics import precision_score, recall_score

In [11]:
rótulos.shape

(100,)

In [2]:


# Suponha que você tenha embeddings de consulta e embeddings de imagens de referência
# Substitua esses dados com seus próprios embeddings
embeddings_de_consulta = np.random.rand(10, 256)  # 10 embeddings de consulta de exemplo (256 dimensões)
embeddings_de_referencia = np.random.rand(100, 256)  # 100 embeddings de referência de exemplo (256 dimensões)

# Suponha que você tenha rótulos de relevância para cada consulta
# Substitua esses dados com seus próprios rótulos de relevância
rótulos_de_relevância = np.random.randint(2, size=(10, 100))  # Matriz de relevância (0 ou 1)

# Número de imagens a serem recuperadas para cada consulta
k = 10

# Recuperação de Imagens
imagens_recuperadas = []
for embedding_de_consulta, rótulos in zip(embeddings_de_consulta, rótulos_de_relevância):
    # Calcula a similaridade entre o embedding de consulta e todos os embeddings de referência
    similaridades = np.dot(embeddings_de_referencia, embedding_de_consulta)

    # Classifica os índices das imagens de referência com base na similaridade
    índices_ordenados = np.argsort(similaridades)[::-1]

    # Recupera as top-k imagens mais similares
    top_k_imagens = índices_ordenados[:k]

    imagens_recuperadas.append(top_k_imagens)

# Validando os Resultados
precision_scores = []
recall_scores = []

for i, (imagens_relevantes, rótulos) in enumerate(zip(imagens_recuperadas, rótulos_de_relevância)):
    # Calcula as métricas de precisão e recall para cada consulta
    precision = precision_score(rótulos, [1 if j in imagens_relevantes else 0 for j in range(len(embeddings_de_referencia))])
    recall = recall_score(rótulos, [1 if j in imagens_relevantes else 0 for j in range(len(embeddings_de_referencia))])

    print(f"Consulta {i+1} - Precisão: {precision}, Recall: {recall}")
    
    precision_scores.append(precision)
    recall_scores.append(recall)

# Calcula a média das métricas de precisão e recall para todas as consultas
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)

print(f"Média de Precisão: {mean_precision}")
print(f"Média de Recall: {mean_recall}")

Consulta 1 - Precisão: 0.4, Recall: 0.08163265306122448
Consulta 2 - Precisão: 0.3, Recall: 0.05263157894736842
Consulta 3 - Precisão: 0.4, Recall: 0.10256410256410256
Consulta 4 - Precisão: 0.3, Recall: 0.061224489795918366
Consulta 5 - Precisão: 0.3, Recall: 0.06382978723404255
Consulta 6 - Precisão: 0.3, Recall: 0.06521739130434782
Consulta 7 - Precisão: 0.4, Recall: 0.08333333333333333
Consulta 8 - Precisão: 0.5, Recall: 0.1
Consulta 9 - Precisão: 0.3, Recall: 0.06521739130434782
Consulta 10 - Precisão: 0.1, Recall: 0.022222222222222223
Média de Precisão: 0.32999999999999996
Média de Recall: 0.06978729497669076


In [1]:
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

# Função para calcular Mean Average Precision@K
def calculate_map_at_k(true_labels, predicted_scores, k):
    # Ordenar as previsões pelos maiores scores
    sorted_indices = np.argsort(predicted_scores)[::-1]
    sorted_labels = true_labels[sorted_indices]

    # Calcular a Precisão Média nos primeiros k elementos
    precision_at_k = np.cumsum(sorted_labels[:k]) / np.arange(1, k + 1)
    map_at_k = np.mean(precision_at_k * sorted_labels[:k])

    return map_at_k

# Função para criar embeddings (substitua isso pela sua lógica real)
def create_embeddings(data):
    # Aqui, use seu modelo de embedding para obter embeddings para os dados
    return np.random.rand(len(data), 64)

# Função para executar consultas e obter scores de similaridade
def run_queries(query_embedding, data_embeddings):
    # Calcular similaridade de cosseno entre a consulta e todos os embeddings
    similarity_scores = cosine_similarity(query_embedding.reshape(1, -1), data_embeddings).flatten()
    return similarity_scores

# Função principal do pipeline de avaliação
def evaluate_retrieval_pipeline(data, queries, k):
    # Criar embeddings para o conjunto de dados
    data_embeddings = create_embeddings(data)

    # Inicializar uma lista para armazenar os resultados MAP@K de cada consulta
    map_at_k_list = []

    # Iterar sobre cada consulta
    for query in queries:
        # Criar embeddings para a consulta
        query_embedding = create_embeddings([query])[0]

        # Executar a consulta para obter scores de similaridade
        similarity_scores = run_queries(query_embedding, data_embeddings)

        # Suponha que as relevâncias verdadeiras são aleatórias para este exemplo
        true_labels = np.random.choice([0, 1], size=len(data), p=[0.9, 0.1])

        # Calcular MAP@K para a consulta atual
        map_at_k = calculate_map_at_k(true_labels, similarity_scores, k)
        map_at_k_list.append(map_at_k)

    # Calcular a média dos resultados MAP@K de todas as consultas
    mean_map_at_k = np.mean(map_at_k_list)

    return mean_map_at_k

# Exemplo de uso
data = np.random.rand(100, 64)  # Dados fictícios
queries = np.random.rand(5, 64)  # Consultas fictícias
k = 10  # Valor de K para MAP@K

result = evaluate_retrieval_pipeline(data, queries, k)
print(f"Mean Average Precision@{k}: {result}")


Mean Average Precision@10: 0.032


In [45]:
import numpy as np

def get_true_labels(array1, array2):
    """
    Compara dois arrays e retorna um novo array de 0s e 1s, indicando se os elementos correspondentes são iguais.

    Parameters:
    - array1: Primeiro array a ser comparado (NumPy array).
    - array2: Segundo array a ser comparado (NumPy array).

    Returns:
    - Novo array contendo 0s e 1s.
    """
    if array1.shape != array2.shape:
        raise ValueError("Os arrays devem ter o mesmo tamanho")

    resultado = np.where(array1 == array2, 1, 0)
    return resultado

# Exemplo de uso com arrays NumPy
array1 = np.array([0, 3, 3, 0, 5, 6])
array2 = np.array([0, 0, 0, 0, 0, 0])

resultado_comparacao = get_true_labels(array1, array2)
print("Resultado da comparação:", resultado_comparacao)

Resultado da comparação: [1 0 0 1 1 1]


In [6]:
import pickle

def load_embeddings(pickle_file_path):
    with open(pickle_file_path, 'rb') as pickle_file:
        loaded_data_dict = pickle.load(pickle_file)

    # Convert lists to numpy arrays
    data = loaded_data_dict["embedding"]
    labels = np.array(loaded_data_dict["target"])
    return data, labels, loaded_data_dict

In [9]:
path_test = './efficientnet_SwaV_test.pickle'
path_train = '../notebooks/efficientnet_SwaV.pickle'

swav_data_test, swav_labels_test, swav_result_test = load_embeddings(path_test)
swav_data_train, swav_labels_train, swav_result_train = load_embeddings(path_train)

In [21]:
print(swav_data_train.shape)
print(swav_data_test.shape)
print(swav_labels_test.shape)

(12131, 1280)
(496, 1280)
(496,)


In [19]:
swav_labels_test[40:70]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [97]:
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

# Dados fictícios

def run_queries(query_embedding, data_embeddings, similarity_function):
    similarity_scores = similarity_function(query_embedding.reshape(1, -1), data_embeddings).flatten()
    sorted_indices = np.argsort(similarity_scores)[::-1]
    return sorted_indices, similarity_scores


def map_relevant_labels(array1, array2):
    """
    Compara dois arrays e retorna um novo array de 0s e 1s, indicando se os elementos correspondentes são iguais.

    Parameters:
    - array1: Primeiro array a ser comparado (NumPy array).
    - array2: Segundo array a ser comparado (NumPy array).

    Returns:
    - Novo array contendo 0s e 1s.
    """
    if array1.shape != array2.shape:
        raise ValueError("Os arrays devem ter o mesmo tamanho")

    result = np.where(array1 == array2, 1, 0)
    return result

def calculate_map_at_k(relevant_labels, k_value):
    ap_num = 0
    for k in range(k_value):
        precision_at_k = sum(relevant_labels[:k+1]) / (k+1)
        # calculate numerator value for ap
        ap_num += precision_at_k * relevant_labels[k]
        # print(f"P@{k+1}_{i+1} = {round(precision_at_k,2)}")

    ap_q = ap_num / k_value

    return ap_q

def evaluate_retrieval_pipeline(
        data_embeddings,
        data_labels, 
        query_embeddings, 
        query_labels, 
        k,
        similarity_function
        ):
    
    map_at_k_list = []
    for q in range(len(query_embeddings)):
        query_label = query_labels[q] 
        sorted_indices, _ = run_queries(query_embeddings[q], data_embeddings, similarity_function)
        sorted_true_labels = data_labels[sorted_indices]
        # [1,0,1,1,1,0] relevant label = 1
        relevant_labels = map_relevant_labels(sorted_true_labels, np.full(len(sorted_true_labels), query_label))
        average_precision = calculate_map_at_k(relevant_labels, k)
        map_at_k_list.append(average_precision)


k_value = 15
queries = 30

map_at_k_list = []


for i in range(queries):

    query_embedding = swav_data_test[i] 
    query_label = swav_labels_test[i] 

    similarity_scores = cosine_similarity(query_embedding.reshape(1, -1), swav_data_train).flatten()
    sorted_indices = np.argsort(similarity_scores)[::-1]
    sorted_true_labels = swav_labels_train[sorted_indices]
    relevant_labels = get_true_labels(sorted_true_labels, np.full(len(sorted_true_labels), query_label))

    ap_num = 0
    for k in range(k_value):

        precision_at_k = sum(relevant_labels[:k+1]) / (k+1)
        # calculate numerator value for ap
        ap_num += precision_at_k * relevant_labels[k]
        # print(f"P@{k+1}_{i+1} = {round(precision_at_k,2)}")

    ap_q = ap_num / k_value
    # print(f"AP@{k+1}: {round(ap_q,2)}") 
    map_at_k_list.append(ap_q)




map_at_k = sum(map_at_k_list) / queries

# generate results
print(f"mAP@{k+1} = {round(map_at_k, 2)}")



mAP@15 = 0.28


In [50]:
sorted_true_labels 
relevant_label = swav_labels_test[0]
get_true_labels(sorted_true_labels, np.full(len(sorted_true_labels), relevant_label))

array([1, 1, 0, ..., 0, 0, 0])

In [82]:
k =1
sum(relevant_labels[:k]) / (k+1)

0.5

In [83]:
relevant_labels[:k]

array([1])

In [77]:
relevant_labels[:k]

array([1, 1, 0, 0, 0, 0, 1, 0, 0])

In [76]:
sorted_true_labels[:k]

array([0, 0, 2, 1, 3, 2, 0, 3, 3], dtype=int64)

In [78]:
query_label

0

In [57]:
# initialize variables
actual = [
    [2, 4, 5, 7],
    [1, 4, 5, 7],
    [5, 8],
    []
]
Q = len(actual)
predicted = [1, 2, 3, 4, 5, 6, 7, 8]
k = 8
ap = []

# loop through and calculate AP for each query q
for q in range(Q):
    ap_num = 0
    # loop through k values
    for x in range(k):
        # calculate precision@k
        act_set = set(actual[q])                                                                                                                                   
        pred_set = set(predicted[:x+1])
        precision_at_k = len(act_set & pred_set) / (x+1)
        # calculate rel_k values
        if predicted[x] in actual[q]:
            rel_k = 1
        else:
            rel_k = 0
        # calculate numerator value for ap
        ap_num += precision_at_k * rel_k
    # now we calculate the AP value as the average of AP
    # numerator values
    ap_q = ap_num / len(actual[q])
    print(f"AP@{k}_{q+1} = {round(ap_q,2)}")
    ap.append(ap_q)

# now we take the mean of all ap values to get mAP
map_at_k = sum(ap) / Q

# generate results
print(f"mAP@{k} = {round(map_at_k, 2)}")

AP@8_1 = 0.54
AP@8_2 = 0.67
AP@8_3 = 0.23


ZeroDivisionError: float division by zero